In [20]:
import streamlit as st
import os
from groq import Groq
import random
import requests
from bs4 import BeautifulSoup
from langchain.chains import ConversationChain, LLMChain
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_community.document_loaders import AsyncChromiumLoader
from langchain_community.document_transformers import BeautifulSoupTransformer

from langchain_core.messages import SystemMessage
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv, find_dotenv
from langchain_community.document_loaders import WikipediaLoader, WebBaseLoader, TextLoader
_ = load_dotenv(find_dotenv()) # read local .env file

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [37]:
import requests
from bs4 import BeautifulSoup

def fetch_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    text_content = ''
    for paragraph in soup.find_all('p'):  # Extract text content from <p> tags
        text_content += paragraph.get_text() + ' '
    return text_content

In [38]:
from langchain.docstore.document import Document

urls = ["https://indianexpress.com/article/india/om-birla-election-lok-sabha-speaker-modi-opposition-9415980/" , "https://www.crummy.com/software/BeautifulSoup/bs4/doc/"]

list_text = []
docs = []
for url in urls:
    doc = Document(page_content=fetch_content(url),
              metadata={
                  "source": url
              }
             )
    docs.append(doc)


In [40]:
print(docs[1].page_content)

Beautiful Soup is a
Python library for pulling data out of HTML and XML files. It works
with your favorite parser to provide idiomatic ways of navigating,
searching, and modifying the parse tree. It commonly saves programmers
hours or days of work. These instructions illustrate all major features of Beautiful Soup 4,
with examples. I show you what the library is good for, how it works,
how to use it, how to make it do what you want, and what to do when it
violates your expectations. This document covers Beautiful Soup version 4.12.2. The examples in
this documentation were written for Python 3.8. You might be looking for the documentation for Beautiful Soup 3.
If so, you should know that Beautiful Soup 3 is no longer being
developed and that all support for it was dropped on December
31, 2020. If you want to learn about the differences between Beautiful
Soup 3 and Beautiful Soup 4, see Porting code to BS4. This documentation has been translated into other languages by
Beautiful Soup us

In [46]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [41]:
chunk_size =1000
chunk_overlap = 150

In [47]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap, 
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator = '\n'
)

In [48]:
c_pages = c_splitter.split_text(docs[0].page_content)
print(len(c_pages))

1


In [49]:
r_pages = r_splitter.split_text(docs[0].page_content)
print(len(r_pages))

7


In [8]:
import re

def clean_text(text):
    # Replace multiple newlines with a single newline
    text = re.sub(r'\n+', '\n', text)
    # Remove leading and trailing whitespaces
    text = text.strip()
    return text

In [9]:
len(docs[0].page_content)

docs[0].page_content = clean_text(docs[0].page_content)

Created a chunk of size 1131, which is longer than the specified 1000
Created a chunk of size 1429, which is longer than the specified 1000
Created a chunk of size 1692, which is longer than the specified 1000


48
45
Page 1:
page_content='Eagle - Wikipedia\nJump to content\nMain menu\nMain menu\nmove to sidebar\nhide\n\t\tNavigation\n\t\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate\n\t\tContribute\n\t\nHelpLearn to editCommunity portalRecent changesUpload file\nSearch\nSearch\nAppearance\nCreate account\nLog in\nPersonal tools\n Create account Log in\n\t\tPages for logged out editors learn more\nContributionsTalk\nContents\nmove to sidebar\nhide\n(Top)\n1\nDescription\n2\nHabitat\n3\nDistribution\n4\nGroups\nToggle Groups subsection\n4.1\nFish eagles\n4.2\nBooted eagles\n4.3\nSnake eagles\n4.4\nHarpy eagles\n5\nSpecies\n6\nIn culture\nToggle In culture subsection\n6.1\nEtymology\n6.2\nReligion and spirituality\n6.3\nHeraldry\n7\nNotes\n8\nReferences\n9\nExternal links\nToggle the table of contents\nEagle\n141 languages' metadata={'source': 'https://en.wikipedia.org/wiki/Eagle', 'title': 'Eagle - Wikipedia', 'language': 'en'}
------------------------------------

In [50]:
cleaned_r_pages = r_splitter.split_documents(docs)
cleaned_c_pages = c_splitter.split_documents(docs)

print(len(cleaned_r_pages))
print(len(cleaned_c_pages))

# Print the cleaned pages
for i, page in enumerate(cleaned_r_pages):
    print(f"Page {i+1}:\n{page}\n{'-'*40}")

89
82
Page 1:
page_content='Lok Sabha MP Om Birla was elected as Lok Sabha Speaker Wednesday by a voice vote for the second time in a row. The Opposition did not press for a division of votes and pro-tem Speaker Bhartruhari Mahtab declared Birla as elected, saying the “ayes have it”. With Birla’s election, Mahtab said, the other motions to propose and second the candidature of K Suresh became infructuous. Prime Minister Narendra Modi and senior Congress leader Rahul Gandhi, whom the Congress has named Leader of the Opposition in the Lok Sabha, congratulated Birla as he took charge. While PM Modi recalled the milestones of his first term as Speaker, Rahul Gandhi sought a voice for the Opposition and Samajwadi Party (SP) leader Akhilesh Yadav said he expects that members will not be suspended for raising their voices in the House. Modi said that Birla was becoming Speaker for the second time after a full term, like Balram Jakhar. He added that such was the work of the Speaker that past S

In [51]:
cleaned_r_pages

[Document(page_content='Lok Sabha MP Om Birla was elected as Lok Sabha Speaker Wednesday by a voice vote for the second time in a row. The Opposition did not press for a division of votes and pro-tem Speaker Bhartruhari Mahtab declared Birla as elected, saying the “ayes have it”. With Birla’s election, Mahtab said, the other motions to propose and second the candidature of K Suresh became infructuous. Prime Minister Narendra Modi and senior Congress leader Rahul Gandhi, whom the Congress has named Leader of the Opposition in the Lok Sabha, congratulated Birla as he took charge. While PM Modi recalled the milestones of his first term as Speaker, Rahul Gandhi sought a voice for the Opposition and Samajwadi Party (SP) leader Akhilesh Yadav said he expects that members will not be suspended for raising their voices in the House. Modi said that Birla was becoming Speaker for the second time after a full term, like Balram Jakhar. He added that such was the work of the Speaker that past Speak

In [52]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma, FAISS

In [53]:
splits = cleaned_r_pages
embedding = OllamaEmbeddings(model="mxbai-embed-large")

vectordb = FAISS.from_documents(
    documents=splits,
    embedding=embedding
)

In [54]:
print(vectordb.index.ntotal)

89


In [65]:
query = "who is om birla"
retriever = vectordb.as_retriever()
docs = retriever.invoke(query)

In [67]:
docs
# Print the cleaned pages
for i, page in enumerate(docs):
    print(f"Page {i+1}:\n{page.metadata}\n{'-'*40}")

Page 1:
{'source': 'https://indianexpress.com/article/india/om-birla-election-lok-sabha-speaker-modi-opposition-9415980/'}
----------------------------------------
Page 2:
{'source': 'https://indianexpress.com/article/india/om-birla-election-lok-sabha-speaker-modi-opposition-9415980/'}
----------------------------------------
Page 3:
{'source': 'https://indianexpress.com/article/india/om-birla-election-lok-sabha-speaker-modi-opposition-9415980/'}
----------------------------------------
Page 4:
{'source': 'https://indianexpress.com/article/india/om-birla-election-lok-sabha-speaker-modi-opposition-9415980/'}
----------------------------------------


In [68]:
vectordb.save_local("faiss_index",)

new_db = FAISS.load_local("faiss_index", embedding, allow_dangerous_deserialization = True)

docs = new_db.similarity_search(query)

In [69]:
for doc in docs:
    print(doc.metadata)

{'source': 'https://indianexpress.com/article/india/om-birla-election-lok-sabha-speaker-modi-opposition-9415980/'}
{'source': 'https://indianexpress.com/article/india/om-birla-election-lok-sabha-speaker-modi-opposition-9415980/'}
{'source': 'https://indianexpress.com/article/india/om-birla-election-lok-sabha-speaker-modi-opposition-9415980/'}
{'source': 'https://indianexpress.com/article/india/om-birla-election-lok-sabha-speaker-modi-opposition-9415980/'}


In [70]:
question = "what is beautifulsoup?"
docs_ss = new_db.similarity_search(question,k=3)
len(docs_ss)

3

In [71]:
for doc in docs_ss:
    print(doc.metadata)

{'source': 'https://www.crummy.com/software/BeautifulSoup/bs4/doc/'}
{'source': 'https://www.crummy.com/software/BeautifulSoup/bs4/doc/'}
{'source': 'https://www.crummy.com/software/BeautifulSoup/bs4/doc/'}


In [72]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)
len(docs_mmr)
for doc in docs_mmr:
    print(doc.metadata)

{'source': 'https://www.crummy.com/software/BeautifulSoup/bs4/doc/'}
{'source': 'https://www.crummy.com/software/BeautifulSoup/bs4/doc/'}
{'source': 'https://www.crummy.com/software/BeautifulSoup/bs4/doc/'}


In [16]:

class Sot_list:

    def sort(self, list1):

        for i in range(len(list1)-1):

            for i in range(len(list1)-1):

                curr = list1[i]
                next = list1[i+1]

                if curr >= next:
                    list1[i] , list1[i+1] = list1[i+1], list1[i]
                else:
                    continue

        return list1
        
    
l = [2,7,4,3,9,1]
sort = Sot_list()
sl = sort.sort(l)
print(sl)

[1, 2, 3, 4, 7, 9]


KeyError: 'GOOGLE_API_KEY'